# PaLM-2 with LangChain

In [ ]:
!pip -q install google-cloud-aiplatform google-api-python-client
!pip -q install langchain google-search-results duckduckgo-search chromadb==0.3.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.7/966.7 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

# Must restart the Notebook runtime if in Colab etc

In [ ]:
#@title Setting up the Auth
import os

from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
import vertexai

PROJECT_ID = ""
vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.235
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Google's PaLM-2 with LangChain


We are using the model in the Chat format

In [ ]:
from google.cloud import aiplatform
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [ ]:
# Standard LLM completion model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat
chat = ChatVertexAI(model='chat-bison',
                    max_output_tokens=256,
                    temperature=0.1,
                    top_p=0.8,
                    top_k=40,)


In [ ]:
messages = [
    HumanMessage(
        content="How do you compare to gpt-4?"
    )
]
chat(messages)

AIMessage(content='I am powered by PaLM 2, which stands for Pathways Language Model 2, a large language model from Google AI.', additional_kwargs={}, example=False)

## Using it as a basic ChatBot with memory and search

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatVertexAI
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Chat
chat_llm = ChatVertexAI(model='chat-bison',
                    max_output_tokens=256,
                    temperature=0.1,
                    top_p=0.8,
                    top_k=40,)


In [ ]:
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]

In [ ]:
agent_chain = initialize_agent(tools,
                               chat_llm,
                               agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                               verbose=True,
                               memory=memory)

In [ ]:
agent_chain.run(input="hi, i am sam")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Hello Sam, how can I help you today?"
}
```

> Finished chain.


'Hello Sam, how can I help you today?'

In [ ]:
agent_chain.run(input="What is Elon Musk's x.ai?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Current Search",
    "action_input": "Elon Musk's x.ai"
}
```
Observation: July 12 (Reuters) - Elon Musk, the billionaire entrepreneur, launched his long-teased artificial intelligence startup xAI on Wednesday, unveiling a team comprised of engineers from the same... Elon Musk has created a new company dedicated to artificial intelligence — and it's called X.AI, as first reported by The Wall Street Journal. The company, which a Nevada filing indicates... What to Know About Elon Musk's New AI Company, xAI Chief Executive Officer of SpaceX and Tesla and owner of Twitter, Elon Musk attends the Viva Technology conference on June 16, 2023 in Paris, France. Elon Musk, CEO of Tesla and SpaceX, and owner of Twitter, on Wednesday announced the debut of a new AI company, xAI, with the goal to "understand the true nature of the universe." According... Elon Musk speaks at an event for SpaceX. (Jonathan Newton/The Washington Post) Elo

"Elon Musk's x.ai is a new artificial intelligence company founded by Elon Musk. The company's goal is to understand the true nature of the universe."

In [ ]:
agent_chain.run(input="Who is on the team?")



> Entering new AgentExecutor chain...
{
    "action": "Current Search",
    "action_input": "Elon Musk's x.ai team"
}
Observation: July 12 (Reuters) - Elon Musk, the billionaire entrepreneur, launched his long-teased artificial intelligence startup xAI on Wednesday, unveiling a team comprised of engineers from the... Elon Musk today (July 12) officially unveiled xAI, an artificial intelligence company he has been teasing for a little while, and its founding members, most of whom he poached from Google,... Elon Musk officially unveiled his new company xAI on Wednesday. Its mission is no less than to probe the "true nature of the universe," according to its website. Musk was hiring for a large... xAI, Elon Musk's newly formed AI company, has revealed itself with a new website detailing its mission and team at https://x.ai/. Musk tweeted the company's intent is to "understand... Dan Hendrycks, who will advise the xAI team, is currently director of the Center for AI Safety, and his work 

"The team at Elon Musk's x.ai is comprised of engineers from Google, OpenAI, and other top AI companies."

In [ ]:
agent_chain.run(input="What do they plan to do?")



> Entering new AgentExecutor chain...
{
    "action": "Current Search",
    "action_input": "Elon Musk's x.ai"
}
Observation: July 12 (Reuters) - Elon Musk, the billionaire entrepreneur, launched his long-teased artificial intelligence startup xAI on Wednesday, unveiling a team comprised of engineers from the same... Elon Musk has created a new company dedicated to artificial intelligence — and it's called X.AI, as first reported by The Wall Street Journal. The company, which a Nevada filing indicates... What to Know About Elon Musk's New AI Company, xAI Chief Executive Officer of SpaceX and Tesla and owner of Twitter, Elon Musk attends the Viva Technology conference on June 16, 2023 in Paris, France. Elon Musk, CEO of Tesla and SpaceX, and owner of Twitter, on Wednesday announced the debut of a new AI company, xAI, with the goal to "understand the true nature of the universe." According... Elon Musk took to Twitter to talk about his new artificial intelligence company, xAI, outlinin

"Elon Musk's x.ai plans to build a rival to ChatGPT."

## Using PaLM2 for Information Retrieval

In [ ]:
# please put your own zip file of text files in here as for copyright reasons I can't share this one
!wget -q -O elon_xai.zip
!unzip -q elon_xai.zip -d xai_articles

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

import time
from typing import List
from pydantic import BaseModel


from langchain.embeddings import VertexAIEmbeddings

In [ ]:
loader = DirectoryLoader('./xai_articles/', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()
len(documents)

7

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

In [ ]:
# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

In [ ]:
text_embedding = embeddings.embed_query(texts[0])
print(f"Your embedding is length {len(text_embedding)}")
print(f"Here's a sample: {text_embedding[:5]}...")

Your embedding is length 768
Here's a sample: [0.008760017342865467, -0.03748807683587074, 0.01045919582247734, 0.03640196844935417, 0.046723198145627975]...


In [ ]:

# embedding_list = embeddings.embed_documents([text.page_content for text in texts])
embeddings

CustomVertexAIEmbeddings(client=<vertexai.language_models._language_models._PreviewTextEmbeddingModel object at 0x7e6fe84ee560>, model_name='textembedding-gecko', temperature=0.0, max_output_tokens=128, top_p=0.95, top_k=40, stop=None, project=None, location='us-central1', credentials=None, request_parallelism=5, max_retries=6, requests_per_minute=100, num_instances_per_batch=5)

### Making a Chroma DB

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
# embedding = embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)


Waiting
....

In [ ]:
# making out retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 7})

In [ ]:
# create the chain to answer questions
# Chat
llm = ChatVertexAI(model='chat-bison',
                    max_output_tokens=256,
                    temperature=0.1,
                    top_p=0.8,
                    top_k=40,)

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
query = "What did Elon announce?"
llm_response = qa_chain(query)
llm_response["result"]

'Elon Musk announced the formation of xAI, a new artificial intelligence company. The goal of xAI is to understand the true nature of the universe.'

In [ ]:
query = "Who is on the team?"
llm_response = qa_chain(query)
llm_response["result"]

'The team is led by Elon Musk, CEO of Tesla and SpaceX. The team also includes Igor Babuschkin, Tony Wu, Christian Szegedy, Greg Yang, Dan Hendrycks, and others.'

In [ ]:
query = "What is he going to call his GPT model?"
llm_response = qa_chain(query)
llm_response["result"]

'Musk has not yet announced the name of his GPT model.'

In [ ]:
query = "Who is going to be ceo?"
llm_response = qa_chain(query)
llm_response["result"]

'Elon Musk is the CEO of xAI.'

In [ ]:
query = "Where is the company going to be based?"
llm_response = qa_chain(query)
llm_response["result"]

'The company will be based in the Bay Area.'

In [ ]:
query = "What did Musk say about China and AI?"
llm_response = qa_chain(query)
print(llm_response["result"])

Musk said that China is interested in working in a cooperative international framework for AI regulation. He added that he had advocated for artificial intelligence regulations and oversight, including in his meetings in China.


## PAL Chain with PaLM2

In [ ]:
from langchain.chains import PALChain
from langchain import OpenAI
from langchain.chains.llm import LLMChain

In [ ]:
# Standard LLM completion model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

In [ ]:
pal_chain = PALChain.from_math_prompt(llm, verbose=True)

In [ ]:
question = "The cafeteria had 23 apples. \
If they used 20 for lunch and bought 6 more,\
how many apples do they have?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """The cafeteria had 23 apples. If they used 20 for lunch and bought 6 more,how many apples do they have?"""
    apples_initial = 23
    apples_used = 20
    apples_bought = 6
    apples_left = apples_initial - apples_used + apples_bought
    result = apples_left
    return result


> Finished chain.


'9'

## GSM8k

In [ ]:
question = "John buys twice as many red ties as blue ties. The red ties cost 50% more than blue ties. \
He spent $200 on blue ties that cost $40 each. How much did he spend on ties?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """John buys twice as many red ties as blue ties. The red ties cost 50% more than blue ties. He spent $200 on blue ties that cost $40 each. How much did he spend on ties?"""
    blue_ties = 200 / 40
    red_ties = blue_ties * 2
    red_ties_cost = red_ties * 40 * 1.5
    total_cost = red_ties_cost + blue_ties * 40
    result = total_cost
    return result


> Finished chain.


'800.0'

In [ ]:
question = "Maggie spent a quarter of her money, while Riza spent one-third of her money. \
They each had $60. How much money do the two of them have left?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """Maggie spent a quarter of her money, while Riza spent one-third of her money. They each had $60. How much money do the two of them have left?"""
    maggie_money = 60
    maggie_spent = maggie_money / 4
    riza_money = 60
    riza_spent = riza_money / 3
    total_spent = maggie_spent + riza_spent
    total_money_left = maggie_money - total_spent
    result = total_money_left
    return result


> Finished chain.


'25.0'

In [ ]:
question = "Maggie and Riza each had $60. Maggie spent a quarter of her money, while Riza spent one-third of her money. \
 How much money do the two of them have left?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """Maggie and Riza each had $60. Maggie spent a quarter of her money, while Riza spent one-third of her money.  How much money do the two of them have left?"""
    maggie_money = 60
    maggie_spent = maggie_money / 4
    riza_money = 60
    riza_spent = riza_money / 3
    maggie_left = maggie_money - maggie_spent
    riza_left = riza_money - riza_spent
    total_left = maggie_left + riza_left
    result = total_left
    return result


> Finished chain.


'85.0'

In [ ]:
question = "In November, a toy was $1000. \
In December, the price increased by 80%. In January, the price decreased by 50%. \
What was the price of the toy after it was discounted in January?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """In November, a toy was $1000. In December, the price increased by 80%. In January, the price decreased by 50%. What was the price of the toy after it was discounted in January?"""
    toy_price_initial = 1000
    toy_price_december = toy_price_initial * 1.8
    toy_price_january = toy_price_december * 0.5
    result = toy_price_january
    return result


> Finished chain.


'900.0'

## Repeat words

In [ ]:
question = "Repeat cheese seven times; every third say whiz"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """Repeat cheese seven times; every third say whiz"""
    for i in range(1, 8):
        if i % 3 == 0:
            print("cheese whiz")
        else:
            print("cheese")

> Finished chain.


'cheese\ncheese\ncheese whiz\ncheese\ncheese\ncheese whiz\ncheese\nNone'

In [ ]:
question = "Say the letters of the alphabet in capital letters, but only the odd ones"
#A C E G I K M O Q S U W Y"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """Say the letters of the alphabet in capital letters, but only the odd ones"""
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    odd_letters = alphabet[1::2]
    result = odd_letters
    return result


> Finished chain.


'BDFHJLNPRTVXZ'